In [1]:
import json
import os as os
import pandas as pd

from collections import OrderedDict
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource, gridplot , output_file
from bokeh.models import HoverTool 
from bokeh.io import vform
from bokeh.palettes import *

import numpy as np

os.chdir('../../Google Drive/elections_niger')

In [2]:
## Données carto
with open('communes_gps.json') as json_data:
    carto_commune = json.load(json_data)
    json_data.close()

In [3]:
communes_ceni = pd.read_csv('data/Niger_Communes.csv')
departements_ceni = pd.read_csv('data/Niger_Departements.csv' , encoding = "ISO-8859-1")
regions_ceni = pd.read_csv('data/Niger_Regions.csv')

In [4]:
full_ceni = pd.merge(departements_ceni , communes_ceni , on = 'ID_DEPART' , how = 'right')
full_ceni = pd.merge(regions_ceni , full_ceni , on = 'ID_REGION' , how = 'right')
full_ceni = full_ceni.drop(['Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','N_COMMUNE','N_DEPART'] , axis = 1)

In [5]:
correction_dictionnary = {'DIFFA': {'KABLEWA': 'KABALEWA',
                                    "N'GOURTI": 'NGOURTI',
                                    "N'GUELBELY": 'NGUEL BELY',
                                    "N'GUIGMI": 'NGUIGMI'},
                          'DOSSO': {"BIRNI N'GAOURE": 'BIRNI NGAOURE',
                                    'DAN-KASSARI': 'DAN KASSARI',
                                    'DOGONKIRIA': 'DOGON KIRYA',
                                    'FALMEY/FALMEY HAOUSSA': 'FALMEY',
                                    'FAREY': 'FARREY',
                                    'GOROUBANKASSAM': 'GOROUN BAKASSAM',
                                    'GUILLADJE': 'GUILADJE',
                                    'KARGUIBANGOU': 'KARGUI BANGOU',
                                    "N'GONGA": 'NGONGA',
                                    'TIBIRI (DOUTCHI)': 'TIBIRI',
                                    'TOMBOKOIREY I': 'TOMBOKOAREY 1',
                                    'TOMBOKOIREY II': 'TOMBOKOAREY 2'},
                          'MARADI': {'ATTANTANE': 'ATTATANE',
                                     'BIRNI LALLE': 'BIRNI NLALLE',
                                     'DAN-GOULBI': 'DAN GOULGI',
                                     'DAN-ISSA': 'DAN ISSA',
                                     'DJIRATAWA': 'DJIRATAOUA',
                                     'EL ALLASSANE MAIREYREY': 'ALHASSANE  MAIREREY',
                                     'GANGARA (AGUIE)': 'GANGARA',
                                     'GUIDAN AMOUMOUNE': 'GUIDAN AMOUMANE',
                                     'KANAN-BAKACHE': 'KANAN BAKACHE',
                                     'MARADI ARRONDISSEMENT 1': 'MARADI I,II,III',
                                     'MARADI ARRONDISSEMENT 2': 'MARADI I,II,III',
                                     'MARADI ARRONDISSEMENT 3': 'MARADI I,II,III',
                                     'SARKIN HAOUSSA': 'SERKIN HAOUSSA',
                                     'SARKIN YAMMA': 'SARKIN YAMA',
                                     'TAGRISS': 'TAGRIS',
                                     'TIBIRI (MARADI)': 'TIBIRI'},
                          'NIAMEY': {'ARRONDISSEMENT 1': 'NIAMEY I',
                                     'ARRONDISSEMENT 2': 'NIAMEY II',
                                     'ARRONDISSEMENT 3': 'NIAMEY III',
                                     'ARRONDISSEMENT 4': 'NIAMEY IV',
                                     'ARRONDISSEMENT 5': 'NIAMEY V'},
                          'TAHOUA': {'ARRONDISSEMENT 1': 'TAHOUA I,II',
                                     'ARRONDISSEMENT 2': 'TAHOUA I,II',
                                     'BABANKATAMI': 'BABAN KATAMI',
                                     "BIRNI N'KONNI": 'BIRNI NKONNI',
                                     'GALMA KOUDAWATCHE': 'GALMA',
                                     'TSERNAOUA': 'TSERNAWA'},
                          'TILLABERI': {'BANIBANGOU': 'BANI BANGOU',
                                        'BANKILARE': 'BANKILLARE',
                                        'DIAGOUROU': 'DIAGOROU',
                                        'DINGAZI': 'DINGAZI BANDA',
                                        'KOURFEYE CENTRE': 'KOURFEY CENTRE',
                                        'KOURTEYE': 'KOURTEY',
                                        'MAKALONDI': 'PARC W', # By elimination
                                        "N'DOUNGA": 'NDOUNGA',
                                        'OURO GUELADJO': 'OURO GUELADIO',
                                        'SAKOIRA': 'SAKOUARA',
                                        'TONDIKIWINDI': 'TONDIKWINDI'},
                          'ZINDER': {'ARRONDISSEMENT 1': 'ZINDER I,II,III,IV,V',
                                     'ARRONDISSEMENT 2': 'ZINDER I,II,III,IV,V',
                                     'ARRONDISSEMENT 3': 'ZINDER I,II,III,IV,V',
                                     'ARRONDISSEMENT 4': 'ZINDER I,II,III,IV,V',
                                     'ARRONDISSEMENT 5': 'ZINDER I,II,III,IV,V',
                                     'DAMAGARAM TAKAYA': 'DAMAGARAM-T',
                                     'DANTCHIAO': 'DAN TCHIAO',
                                     'DOGO-DOGO': 'DOGO DOGO',
                                     'GANGARA (TANOUT)': 'GANGARA',
                                     'MATAMEY': 'MATAMEYE'}}

In [6]:
full_ceni['commune_to_match'] = full_ceni['NOM_COMMUNE']

def correct_name_commune(data , dictionnary):
    for com in dictionnary.keys() :
        data[data == com] = dictionnary[com]
    return data

for reg in correction_dictionnary.keys() :
    full_ceni.loc[full_ceni.NOM_REGION == reg , 
                  'commune_to_match'] = correct_name_commune(full_ceni.commune_to_match[full_ceni.NOM_REGION == reg] , 
                                                                  correction_dictionnary[reg])

In [7]:
regions_names = []
regions_ID = []
xs = []
ys = []
values = []
cols = []

def match_communes(ceni_to_match , commune_data):
    name = pd.Series(commune_data['nom']).str.replace('É','E')
    if name.isin(ceni_to_match.commune_to_match)[0] == True:
        name = name[0]
        commune_data['ceni_ID'] = str(ceni_to_match.ID_COMMUNE[full_ceni.commune_to_match ==
                                                           name][ceni_to_match.ID_COMMUNE[full_ceni.commune_to_match == 
                                                                                      name].index[0]])
    else :
        commune_data['ceni_ID'] = ''
    return(commune_data)

In [8]:
df = pd.DataFrame(carto_commune ).transpose().reset_index()
for reg in df.region.unique() :
    ids = df[df.region == reg ]['index']
    ceni_to_match = full_ceni[full_ceni.NOM_REGION == reg]
    for ID in ids.index:
        carto_commune[str(ids[ID])] = match_communes(ceni_to_match , carto_commune[str(ids[ID])])

In [9]:
non_matched_gps = {}
for i in carto_commune.keys():
    if carto_commune[i]['ceni_ID'] == '' :
        non_matched_gps[i] = carto_commune[i]
non_matched_gps = pd.DataFrame(non_matched_gps).transpose()

In [10]:
df2 = pd.DataFrame(carto_commune ).transpose().reset_index()
non_matched_ceni = full_ceni[~(full_ceni.ID_COMMUNE.astype(str).isin(df2.ceni_ID))]
non_matched_ceni = non_matched_ceni[non_matched_ceni.ID_DEPART < 900]

In [11]:
non_matched_ceni

,ID_REGION,NOM_REGION,ID_DEPART,NOM_DEPART,ID_COMMUNE,NOM_COMMUNE,commune_to_match
115,4,MARADI,490,MARADI,49002,MARADI ARRONDISSEMENT 2,"MARADI I,II,III"
116,4,MARADI,490,MARADI,49003,MARADI ARRONDISSEMENT 3,"MARADI I,II,III"
160,5,TAHOUA,590,TAHOUA,59002,ARRONDISSEMENT 2,"TAHOUA I,II"
257,7,ZINDER,790,ZINDER,79002,ARRONDISSEMENT 2,"ZINDER I,II,III,IV,V"
258,7,ZINDER,790,ZINDER,79003,ARRONDISSEMENT 3,"ZINDER I,II,III,IV,V"
259,7,ZINDER,790,ZINDER,79004,ARRONDISSEMENT 4,"ZINDER I,II,III,IV,V"
260,7,ZINDER,790,ZINDER,79005,ARRONDISSEMENT 5,"ZINDER I,II,III,IV,V"


In [12]:
store_electeurs = pd.HDFStore('data/full_data.h5')
data_electeurs = store_electeurs['complete_data']
store_electeurs.close()

In [13]:
data_electeurs.NOM_COMMUNE[data_electeurs.ID_COMMUNE.isin([49002,49003]) ] ='MARADI ARRONDISSEMENT 1'
data_electeurs.ID_COMMUNE[data_electeurs.ID_COMMUNE.isin([49002,49003]) ] = 49001

data_electeurs.NOM_COMMUNE[data_electeurs.ID_COMMUNE.isin([59002]) ] ='ARRONDISSEMENT 1'
data_electeurs.ID_COMMUNE[data_electeurs.ID_COMMUNE.isin([59002]) ] = 59001

data_electeurs.NOM_COMMUNE[data_electeurs.ID_COMMUNE.isin([79002,79003,79004,79005]) ] ='ARRONDISSEMENT 1'
data_electeurs.ID_COMMUNE[data_electeurs.ID_COMMUNE.isin([79002,79003,79004,79005]) ] = 79001

C:\Users\grlurton\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\grlurton\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\grlurton\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\

In [14]:
elec_commune = data_electeurs.groupby('ID_COMMUNE').apply(len)

In [15]:
def create_binning(data , variable , n_colors , index) :
    data = data.reset_index()
    bins = np.linspace(data[variable].min(), data[variable].max(), n_colors)
    data['bin'] = np.digitize(data[variable], bins) - 1
    out = data.set_index(index)
    return out

In [16]:
elec_commune = pd.DataFrame(elec_commune).reset_index()

In [17]:
elec_commune.columns = ['ID_COMMUNE' , 'n_electeurs']

In [18]:
if YlOrRd9[0] == '#800026':
    YlOrRd9.reverse()
col1 = YlOrRd9
n_colors = len(col1)
bins = np.linspace(elec_commune['n_electeurs'].min(), elec_commune['n_electeurs'].max(), n_colors)
elec_commune['bin'] = np.digitize(elec_commune['n_electeurs'], bins) - 1
elec_commune = elec_commune.set_index('ID_COMMUNE')

In [46]:
elec_commune.head()

,n_electeurs,bin
ID_COMMUNE,,
10101,33.138946,4
10201,34.138261,5
10202,33.138946,4
10203,32.139630,4
10301,36.139630,7


In [20]:
for i in carto_commune.keys() :
    reg_sel = carto_commune[i]
    ceni_ID = reg_sel['ceni_ID']
    carto_commune[i]['n_electeurs'] = elec_commune['n_electeurs'][int(ceni_ID)]
    carto_commune[i]['color'] = col1[elec_commune['bin'][int(ceni_ID)]]

In [21]:
regions_names = []
regions_ID = []
xs = []
ys = []
values = []
cols = []
n_elect = []
colors = []

for i in carto_commune.keys() :
    reg_sel = carto_commune[i]
    name = reg_sel['nom']
    ID = reg_sel['ID']
    x = reg_sel['coordinates']['x']
    y = reg_sel['coordinates']['y']
    n_electeurs = reg_sel['n_electeurs']
    col = reg_sel['color']
    
    regions_names.append(name)
    regions_ID.append(ID)
    xs.append(x)
    ys.append(y)
    n_elect.append(n_electeurs)
    colors.append(col)
    
map_source = ColumnDataSource(
    data = dict(
        x=xs,
        y=ys,
        ID=regions_ID,
        name=regions_names ,
        n_electeurs = n_elect ,
        color = colors))

In [22]:
output_file('Commune_voters.html')

p = figure(#plot_width=700, plot_height=700,
           title="Registered voters by Commune" , title_text_font_size='12pt' ,
           tools='wheel_zoom,hover')
p.patches('x', 'y', 
          fill_color = "color" ,
          line_color="grey", line_width=0.5,
          source=map_source)

hover = p.select(dict(type=HoverTool))
hover.point_policy = "follow_mouse"
hover.tooltips = OrderedDict([
        ("Nom", "@name") ,
        ("N Registered Voters" , "@n_electeurs")])

show(p)

In [42]:
def median_age(data):
    med = data.age.median()
    return med

age_commune = data_electeurs.groupby('ID_COMMUNE').apply(median_age)

In [56]:
elec_commune = pd.DataFrame(age_commune).reset_index()
elec_commune.columns = ['ID_COMMUNE' , 'n_electeurs']

In [57]:
YlOrRd7.reverse()
col1 = YlOrRd7
n_colors = len(col1)
bins = np.linspace(elec_commune['n_electeurs'].min(), elec_commune['n_electeurs'].max(), n_colors)
elec_commune['bin'] = np.digitize(elec_commune['n_electeurs'], bins) - 1
elec_commune = elec_commune.set_index('ID_COMMUNE')

In [58]:
for i in carto_commune.keys() :
    reg_sel = carto_commune[i]
    ceni_ID = reg_sel['ceni_ID']
    carto_commune[i]['n_electeurs'] = elec_commune['n_electeurs'][int(ceni_ID)]
    carto_commune[i]['color'] = col1[elec_commune['bin'][int(ceni_ID)]]

In [59]:
regions_names = []
regions_ID = []
xs = []
ys = []
values = []
cols = []
n_elect = []
colors = []

for i in carto_commune.keys() :
    reg_sel = carto_commune[i]
    name = reg_sel['nom']
    ID = reg_sel['ID']
    x = reg_sel['coordinates']['x']
    y = reg_sel['coordinates']['y']
    n_electeurs = reg_sel['n_electeurs']
    col = reg_sel['color']
    
    regions_names.append(name)
    regions_ID.append(ID)
    xs.append(x)
    ys.append(y)
    n_elect.append(n_electeurs)
    colors.append(col)
    
map_source = ColumnDataSource(
    data = dict(
        x=xs,
        y=ys,
        ID=regions_ID,
        name=regions_names ,
        n_electeurs = n_elect ,
        color = colors))

In [60]:
output_file('Commune_voters.html')

p = figure(#plot_width=700, plot_height=700,
           title="Registered voters by Commune" , title_text_font_size='12pt' ,
           tools='wheel_zoom,hover')
p.patches('x', 'y', 
          fill_color = "color" ,
          line_color="grey", line_width=0.5,
          source=map_source)

hover = p.select(dict(type=HoverTool))
hover.point_policy = "follow_mouse"
hover.tooltips = OrderedDict([
        ("Nom", "@name") ,
        ("N Registered Voters" , "@n_electeurs")])

show(p)